# Benjamin Wilke
# Homework 3

# Question 1

Compare your given name with your nickname (if you don’t have a nickname, invent one for this assignment) by answering the following questions:

What is the edit distance between your nickname and your given name?

What is the percentage string match between your nickname and your given name?

Show your work for both calculations.

In [63]:
nick = "Ben"
full = "Benjamin"

"Benjamin" - "Ben" = "jamin", which is length 5

In [65]:
# https://towardsdatascience.com/calculating-string-similarity-in-python-276e18a7d33a
import Levenshtein
Levenshtein.distance(nick, full)

5

Percentage string match is (3 / 8) * 100 = 37.5%

# Question 2

Find a friend (or family member or classmate) who you know has read a certain book. Without your friend knowing, copy the first two sentences of that book. Now rewrite the words from those sentences, excluding stop words. Now tell your friend to guess which book the words are from by reading them just that list of words.

Did you friend correctly guess the book on the first try?

What did he or she guess?

Explain why you think you friend either was or was not able to guess the book from hearing the list of words.

In [66]:
# The first two sentences from Jurrasic Park
jurassic_park = "The late twentieth century has witnessed a scientific gold rush of astonishing proportions: the \
headlong and furious haste to commercialize genetic engineering. This enterprise has proceeded \
so rapidly - with so little outside commentary - that its dimensions and implications are hardly \
understood at all."

In [67]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

stop_words = set(stopwords.words('english'))
word_tokens = [w for w in word_tokenize(jurassic_park) if not w in string.punctuation]
filtered_sentence = [w for w in word_tokens if not w in stop_words]

print("Tokenized Source Text \n")
print(word_tokens)

Tokenized Source Text 

['The', 'late', 'twentieth', 'century', 'has', 'witnessed', 'a', 'scientific', 'gold', 'rush', 'of', 'astonishing', 'proportions', 'the', 'headlong', 'and', 'furious', 'haste', 'to', 'commercialize', 'genetic', 'engineering', 'This', 'enterprise', 'has', 'proceeded', 'so', 'rapidly', 'with', 'so', 'little', 'outside', 'commentary', 'that', 'its', 'dimensions', 'and', 'implications', 'are', 'hardly', 'understood', 'at', 'all']


In [68]:
print("Tokenized Text with Stop Words Removed \n")
print(filtered_sentence)

Tokenized Text with Stop Words Removed 

['The', 'late', 'twentieth', 'century', 'witnessed', 'scientific', 'gold', 'rush', 'astonishing', 'proportions', 'headlong', 'furious', 'haste', 'commercialize', 'genetic', 'engineering', 'This', 'enterprise', 'proceeded', 'rapidly', 'little', 'outside', 'commentary', 'dimensions', 'implications', 'hardly', 'understood']


My friend did actually guess the book correctly with a subtle hint that it was written by Michael Crichton. Although, Westworld may have also been a good candidate.

I think he was able to guess based on the inclusion of several key words, which would leave one to believe these sentences are about genetic engineering and the repercussions of hasty scientific pursuit.

# Question 3

Run one of the stemmers available in Python.

Run the same two sentences from question 2 above through the stemmer and show the results.

How many of the outputted stems are valid morphological roots of the corresponding words?

Express this answer as a percentage.

In [69]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [70]:
stems = (list(zip([ps.stem(w) for w in word_tokens], word_tokens)))
print(stems)

[('the', 'The'), ('late', 'late'), ('twentieth', 'twentieth'), ('centuri', 'century'), ('ha', 'has'), ('wit', 'witnessed'), ('a', 'a'), ('scientif', 'scientific'), ('gold', 'gold'), ('rush', 'rush'), ('of', 'of'), ('astonish', 'astonishing'), ('proport', 'proportions'), ('the', 'the'), ('headlong', 'headlong'), ('and', 'and'), ('furiou', 'furious'), ('hast', 'haste'), ('to', 'to'), ('commerci', 'commercialize'), ('genet', 'genetic'), ('engin', 'engineering'), ('thi', 'This'), ('enterpris', 'enterprise'), ('ha', 'has'), ('proceed', 'proceeded'), ('so', 'so'), ('rapidli', 'rapidly'), ('with', 'with'), ('so', 'so'), ('littl', 'little'), ('outsid', 'outside'), ('commentari', 'commentary'), ('that', 'that'), ('it', 'its'), ('dimens', 'dimensions'), ('and', 'and'), ('implic', 'implications'), ('are', 'are'), ('hardli', 'hardly'), ('understood', 'understood'), ('at', 'at'), ('all', 'all')]


In [71]:
badstem = [3,4,5,7,11,12,16,17,19,20,21,22,23,25,27,30,31,32,35,37,39]

print("There are {} invalid output stems. \n".format(len(badstem)))

for idx, word in enumerate(stems):
    if idx in badstem:
        print(word)        

print('\n The percentage of valid stems is {}%'.format(((len(stems) - len(badstem)) / len(stems)) * 100))

There are 21 invalid output stems. 

('centuri', 'century')
('ha', 'has')
('wit', 'witnessed')
('scientif', 'scientific')
('astonish', 'astonishing')
('proport', 'proportions')
('furiou', 'furious')
('hast', 'haste')
('commerci', 'commercialize')
('genet', 'genetic')
('engin', 'engineering')
('thi', 'This')
('enterpris', 'enterprise')
('proceed', 'proceeded')
('rapidli', 'rapidly')
('littl', 'little')
('outsid', 'outside')
('commentari', 'commentary')
('dimens', 'dimensions')
('implic', 'implications')
('hardli', 'hardly')

 The percentage of valid stems is 51.162790697674424%


# Question 4

Run an RNN with Pre-Trained and Self-Trained Vectors on the IMDB dataset.

Do your self-trained vectors show the same behavior as pre-trained (King - man + Woman = Queen)

How much do the pretrained vectors change when you allow them to be updated?

In [1]:
import scipy
from scipy.spatial import distance
import spacy
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

In [2]:
vocabulary_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size) # num_words: integer or None. Top most frequent words to consider. Any less frequent word will appear as oov_char value in the sequence data.

print("Loaded Train: {}".format(len(X_train)))
print("Loaded Test: {}".format(len(X_test)))    # 0 is negative sentiment, 1 is positive

Loaded Train: 25000
Loaded Test: 25000


In [3]:
# get dictionary word to integer index
word_index = imdb.get_word_index()

In [4]:
# shift integer + 3 to make way for reserved indices
word_index = {k:(v + 3) for k,v in word_index.items()}
# insert special token to integers into our dictionary
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

# create reverse index mapping to word dict
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [5]:
# pad all sequences to be common length, or chop those that extend 
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

## Train Model with Self-Trained Word Embeddings

In [6]:
embedding_size = 100
model_self = Sequential()
model_self.add(Embedding(vocabulary_size, embedding_size, input_length=max_words)) # <-- layer accepts word numbers, instead of one-hot vectors
model_self.add(LSTM(256, unroll=True))
model_self.add(Dense(1, activation="sigmoid"))
print(model_self.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 100)          500000    
_________________________________________________________________
lstm (LSTM)                  (None, 256)               365568    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 865,825
Trainable params: 865,825
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
model_self.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# define checkpoint callback
checkpoint_self = ModelCheckpoint('model_self.h5', monitor='accuracy', verbose=1, save_best_only=True, mode='max')

In [9]:
model_self.fit(X_train, y_train, batch_size=100, epochs=5, callbacks=[checkpoint_self])

Train on 25000 samples
Epoch 1/5
24900/25000 [============================>.] - ETA: 2s - loss: 0.5074 - accuracy: 0.7471
Epoch 00001: accuracy improved from -inf to 0.74756, saving model to model_self.h5
25000/25000 [==============================] - 510s 20ms/sample - loss: 0.5068 - accuracy: 0.7476
Epoch 2/5
24900/25000 [============================>.] - ETA: 1s - loss: 0.3792 - accuracy: 0.8368
Epoch 00002: accuracy improved from 0.74756 to 0.83636, saving model to model_self.h5
25000/25000 [==============================] - 462s 18ms/sample - loss: 0.3797 - accuracy: 0.8364
Epoch 3/5
24900/25000 [============================>.] - ETA: 1s - loss: 0.3160 - accuracy: 0.8677
Epoch 00003: accuracy improved from 0.83636 to 0.86764, saving model to model_self.h5
25000/25000 [==============================] - 493s 20ms/sample - loss: 0.3162 - accuracy: 0.8676
Epoch 4/5
24900/25000 [============================>.] - ETA: 1s - loss: 0.3142 - accuracy: 0.8698
Epoch 00004: accuracy improved f

In [18]:
scores = model_self.evaluate(X_test, t_test, verbose=0)
print("Test Set Accuracy: {}".format(scores[1]*100))

Test Set Accuracy: 86.06799840927124


## Train Model with PreTrained Word Embeddings (GloVe Fixed)

In [19]:
# using GloVe vectors
embeddings_index = dict()
f = open('WilkeRevEngineering/glove.6B/glove.6B.100d.txt', 'r', encoding="UTF-8")
# is in format: the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953.....
for line in f:
    values = line.split()                              #<-- split each line on spaces
    word = values[0]                                   #<-- gets word from first position
    coefs = np.asarray(values[1:], dtype='float32')    #<-- gets coefs from every position after 1, convert to np array
    embeddings_index[word] = coefs                     #<-- add to dict() word is key, coefs are np array
f.close()

In [21]:
embedding_matrix = np.zeros((vocabulary_size, 100))       #<-- initialize empty (0) np array of shape 5000 x 100
index = 0                                                 #<-- initialize index
for word in word_index:                                   #<-- for each word
    index = word_index[word]                              #<-- get index
    if index > vocabulary_size - 1:                       #<-- IMDB is sorted by word count frequencies HOW CONVENIENT!
        pass                                              #<-- outside of vocabulary size
    else:                                                 #<-- inside of vocabulary size
        embedding_vector = embeddings_index.get(word)     #<-- retrieve embedding coeffecients by word lookup
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector    #<-- set row of embedding matrix to embedding coeffecients (if exists)
embedding_matrix[2] = embeddings_index.get('unk')
embedding_matrix[1] = np.ones(100)

In [22]:
model_glove_fix = Sequential()
model_glove_fix.add(Embedding(vocabulary_size, embedding_size, weights=[embedding_matrix], input_length=max_words, trainable=False))
model_glove_fix.add(LSTM(256, unroll=True))
model_glove_fix.add(Dense(1, activation="sigmoid"))
print(model_glove_fix.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 865,825
Trainable params: 365,825
Non-trainable params: 500,000
_________________________________________________________________
None


In [23]:
model_glove_fix.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
# define checkpoint callback
checkpoint_glove_fix = ModelCheckpoint('model_glove_fix.h5', monitor='accuracy', verbose=1, save_best_only=True, mode='max')

In [25]:
model_glove_fix.fit(X_train, y_train, batch_size=100, epochs=5, callbacks=[checkpoint_glove_fix])

Train on 25000 samples
Epoch 1/5
24900/25000 [============================>.] - ETA: 1s - loss: 0.6209 - accuracy: 0.6612
Epoch 00001: accuracy improved from -inf to 0.66136, saving model to model_glove_fix.h5
25000/25000 [==============================] - 493s 20ms/sample - loss: 0.6207 - accuracy: 0.6614
Epoch 2/5
24900/25000 [============================>.] - ETA: 1s - loss: 0.5290 - accuracy: 0.7385
Epoch 00002: accuracy improved from 0.66136 to 0.73856, saving model to model_glove_fix.h5
25000/25000 [==============================] - 397s 16ms/sample - loss: 0.5288 - accuracy: 0.7386
Epoch 3/5
24900/25000 [============================>.] - ETA: 2s - loss: 0.4168 - accuracy: 0.8138
Epoch 00003: accuracy improved from 0.73856 to 0.81356, saving model to model_glove_fix.h5
25000/25000 [==============================] - 571s 23ms/sample - loss: 0.4169 - accuracy: 0.8136
Epoch 4/5
24900/25000 [============================>.] - ETA: 1s - loss: 0.3844 - accuracy: 0.8321
Epoch 00004: accu

In [26]:
scores = model_glove_fix.evaluate(X_test, t_test, verbose=0)
print("Test Set Accuracy: {}".format(scores[1]*100))

Test Set Accuracy: 86.95600032806396


## Train Model with PreTrained Word Embeddings (GloVe Fine Tune)

In [32]:
model_glove_finetune = Sequential()
model_glove_finetune.add(Embedding(vocabulary_size, embedding_size, weights=[embedding_matrix], input_length=max_words, trainable=True))
model_glove_finetune.add(LSTM(256, unroll=True))
model_glove_finetune.add(Dense(1, activation="sigmoid"))
print(model_glove_finetune.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 100)          500000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 865,825
Trainable params: 865,825
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
model_glove_finetune.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
checkpoint_glove_finetune = ModelCheckpoint('model_glove_finetune.h5', monitor='accuracy', verbose=1, save_best_only=True, mode='max')

In [35]:
model_glove_finetune.fit(X_train, y_train, batch_size=100, epochs=5, callbacks=[checkpoint_glove_finetune])

Train on 25000 samples
Epoch 1/5
24900/25000 [============================>.] - ETA: 2s - loss: 0.5050 - accuracy: 0.7447
Epoch 00001: accuracy improved from -inf to 0.74504, saving model to model_glove_finetune.h5
25000/25000 [==============================] - 501s 20ms/sample - loss: 0.5046 - accuracy: 0.7450
Epoch 2/5
24900/25000 [============================>.] - ETA: 1s - loss: 0.3877 - accuracy: 0.8303
Epoch 00002: accuracy improved from 0.74504 to 0.83028, saving model to model_glove_finetune.h5
25000/25000 [==============================] - 443s 18ms/sample - loss: 0.3877 - accuracy: 0.8303
Epoch 3/5
24900/25000 [============================>.] - ETA: 2s - loss: 0.2922 - accuracy: 0.8820
Epoch 00003: accuracy improved from 0.83028 to 0.88192, saving model to model_glove_finetune.h5
25000/25000 [==============================] - 619s 25ms/sample - loss: 0.2920 - accuracy: 0.8819
Epoch 4/5
24900/25000 [============================>.] - ETA: 1s - loss: 0.2290 - accuracy: 0.9103
Ep

In [36]:
scores = model_glove_finetune.evaluate(X_test, t_test, verbose=0)
print("Test Set Accuracy: {}".format(scores[1]*100))

Test Set Accuracy: 87.56399750709534


## Word Embedding Comparison Functions

In [37]:
embedding_self = model_self.get_weights()[0]
embedding_glove_fix = model_glove_fix.get_weights()[0]
embedding_glove_finetune = model_glove_finetune.get_weights()[0]

In [38]:
def getVectorSelfTrain(word):
    try:
        index = word_index[word]
        return embedding_self[index]
    except KeyError:
        print("Word not in vocabulary")

def getVectorGloVeFix(word):
    try:
        index = word_index[word]
        return embedding_glove_fix[index]
    except KeyError:
        print("Word not in vocabulary")

def getVectorGloVeTune(word):
    try:
        index = word_index[word]
        return embedding_glove_finetune[index]
    except KeyError:
        print("Word not in vocabulary")

In [43]:
# The Cosine Similarity values for different documents, 1 (same direction), 0 (90 deg.), -1 (opposite directions).
# Cosine Distance, 1 - Cosine Similarity, then can take on values from 0 to 2 
def cosineDistanceAnalysis(word):
    try:
        index = word_index[word]
        print("Distance between Self and GloVe Freeze: {}".format(distance.cosine(embedding_self[index], embedding_glove_fix[index])))
        print("Distance between Self and GloVe Tune: {}".format(distance.cosine(embedding_self[index], embedding_glove_finetune[index])))
        print("Distance between GloVe Freeze and GloVe Tune: {}".format(distance.cosine(embedding_glove_fix[index], embedding_glove_finetune[index])))
    except KeyError:
        print("Word not in vocabulary")

## Do your self-trained vectors show the same behavior as pre-trained (king - man + woman = queen)

In [46]:
queenselfhat = getVectorSelfTrain("king") - getVectorSelfTrain("man") + getVectorSelfTrain("woman")
distance.cosine(queenselfhat, getVectorSelfTrain("queen"))

1.4907889366149902

In [48]:
queenglovefixhat = getVectorGloVeFix("king") - getVectorGloVeFix("man") + getVectorGloVeFix("woman")
distance.cosine(queenglovefixhat, getVectorGloVeFix("queen"))

0.21655863523483276

In [49]:
queenglovetunehat = getVectorGloVeTune("king") - getVectorGloVeTune("man") + getVectorGloVeTune("woman")
distance.cosine(queenglovetunehat, getVectorGloVeTune("queen"))

0.2385583519935608

The above exhibits that my self-trained embeddings perform poorly for learning (king - man + woman = queen). The calculated vector and actual vector are quite different. The pre-trained GloVe embeddings show the opposite. They are both able to get pretty close to the actual queen vector when calculating an estimate from (king - man + woman). You'll note that the fine-tuned GloVe embeddings are not as close as the fixed GloVe embeddings, which makes sense empirically if the self-trained embeddings are having trouble learning this relationship, then introducing the IMDB data to fine tune the GloVe embeddings will make them worse.

In [60]:
queenselfhat = getVectorSelfTrain("paris") - getVectorSelfTrain("france") + getVectorSelfTrain("britain")
distance.cosine(queenselfhat, getVectorSelfTrain("london"))

1.505023717880249

In [61]:
queenglovefixhat = getVectorGloVeFix("paris") - getVectorGloVeFix("france") + getVectorGloVeFix("britain")
distance.cosine(queenglovefixhat, getVectorGloVeFix("london"))

0.11206841468811035

In [62]:
queenglovetunehat = getVectorGloVeTune("paris") - getVectorGloVeTune("france") + getVectorGloVeTune("britain")
distance.cosine(queenglovetunehat, getVectorGloVeTune("london"))

0.1295989751815796

This also holds true for learning (paris - france + britain = london). My self-trained embeddings had a hard time learning this relationship, which sounds right considering they are movie reviews! The GloVe embeddings did very well learning this relationship as the estimated vector is close to the actual vector. The same paradigm holds for the fine-tuned GloVe as above - by allowing a dataset with less data supporting these relationship to update the weights the embeddings lose some of this context. This is why the fine-tuned embeddings become worse than keeping them fixed for this scenario.

## How much do the pretrained vectors change when you allow them to be updated?

In [44]:
cosineDistanceAnalysis("dog")

Distance between Self and GloVe Freeze: 1.0866677090525627
Distance between Self and GloVe Tune: 1.0833083391189575
Distance between GloVe Freeze and GloVe Tune: 0.0014770030975341797


In [50]:
cosineDistanceAnalysis("man")

Distance between Self and GloVe Freeze: 0.8358397483825684
Distance between Self and GloVe Tune: 0.8344792127609253
Distance between GloVe Freeze and GloVe Tune: 0.0012821555137634277


In [51]:
cosineDistanceAnalysis("woman")

Distance between Self and GloVe Freeze: 0.9740357659757137
Distance between Self and GloVe Tune: 0.9748082738369703
Distance between GloVe Freeze and GloVe Tune: 0.004621982574462891


In [52]:
cosineDistanceAnalysis("building")

Distance between Self and GloVe Freeze: 0.8721013516187668
Distance between Self and GloVe Tune: 0.8739651590585709
Distance between GloVe Freeze and GloVe Tune: 0.0006364583969116211


In [53]:
cosineDistanceAnalysis("professor")

Distance between Self and GloVe Freeze: 1.0551324374973774
Distance between Self and GloVe Tune: 1.0634262785315514
Distance between GloVe Freeze and GloVe Tune: 0.006900012493133545


Actually - not much. This is evident in the distance between GloVe Freeze and GloVe Tune vectors for each of the sample words: dog, man, woman, building, and professor above. 